# 07. Genetic Programming (GP) 實驗

## 📖 實驗目標

在前面的實驗中，我們測試了多種模型：
- ✅ **Logistic Regression**: 高血糖/高血脂表現最佳 (AUC 0.93/0.89)
- ✅ **XGBoost**: 平衡效能與可解釋性的通用選擇
- ✅ **ANN**: 高血壓表現最佳 (AUC 0.803)
- ✅ **SVM**: 穩定的中階表現 (11 秒訓練完成)

本 notebook 將測試 **Genetic Programming (GP)**，理由：
1. 🧬 **符號回歸**: 自動發現數學公式，最高可解釋性
2. 🔍 **特徵交互發現**: 能找到人類難以發現的特徵組合
3. 🎓 **指導教授專長**: 指導教授專精 GP，值得深入探索
4. 📊 **非線性建模**: 不假設模型形式，從資料中演化出最佳公式

**⚠️ 注意**：
- GP 訓練**非常慢**（需要演化多個世代，可能需要數小時）
- 容易過擬合（公式可能過於複雜）
- 需要控制公式複雜度（max_depth, parsimony_coefficient）

---

## 📋 實驗流程

1. 載入資料（使用與 03-06 相同的資料處理）
2. 使用 **gplearn** 套件進行符號回歸
3. 分析演化出的公式
4. 與最佳模型比較
5. 提取特徵重要性與交互作用

## 0. 安裝必要套件

⚠️ **首次執行前請先安裝 gplearn**

In [ ]:
# 安裝 gplearn 套件 (如果尚未安裝)
import sys
!{sys.executable} -m pip install gplearn

print("✅ gplearn 安裝完成！")

---

## 1. 載入套件與資料

In [2]:
# 基礎套件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time

# 機器學習套件
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix
)
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

# Genetic Programming 套件
try:
    from gplearn.genetic import SymbolicClassifier
    print("✅ gplearn 載入成功")
except ImportError:
    print("❌ gplearn 未安裝")
    print("請執行: pip install gplearn")
    raise

# 設定
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 套件載入完成")

❌ gplearn 未安裝
請執行: pip install gplearn


ModuleNotFoundError: No module named 'gplearn'

In [ ]:
# 載入資料
data_path = Path('../../data/processed/SUA_CVDs_wide_format.csv')
df = pd.read_csv(data_path)

print("✅ 資料載入成功！")
print(f"資料形狀: {df.shape[0]:,} 人, {df.shape[1]} 個欄位")

## 2. 準備特徵和目標變數

In [ ]:
# 定義特徵組
demographic_features = ['sex', 'Age']
biomarker_names = ['FBG', 'TC', 'Cr', 'UA', 'GFR', 'BMI', 'SBP', 'DBP']
t1_features = [f'{name}_T1' for name in biomarker_names]
t2_features = [f'{name}_T2' for name in biomarker_names]
delta1_features = [f'Delta1_{name}' for name in biomarker_names]

# 完整特徵集
X_columns = demographic_features + t1_features + t2_features + delta1_features
X = df[X_columns]

# 目標變數（轉換為 0/1）
y_hypertension = (df['hypertension_T3'] == 2).astype(int)
y_hyperglycemia = (df['hyperglycemia_T3'] == 2).astype(int)
y_dyslipidemia = (df['dyslipidemia_T3'] == 2).astype(int)

print(f"特徵數: {len(X_columns)} 個")
print(f"\n目標變數分佈:")
print(f"  高血壓患病率: {y_hypertension.mean():.2%}")
print(f"  高血糖患病率: {y_hyperglycemia.mean():.2%}")
print(f"  高血脂患病率: {y_dyslipidemia.mean():.2%}")

## 3. 資料分割與標準化

In [ ]:
# 資料分割
X_train, X_test, y_train_hp, y_test_hp = train_test_split(
    X, y_hypertension, test_size=0.2, random_state=42, stratify=y_hypertension
)

_, _, y_train_hg, y_test_hg = train_test_split(
    X, y_hyperglycemia, test_size=0.2, random_state=42, stratify=y_hypertension
)

_, _, y_train_dl, y_test_dl = train_test_split(
    X, y_dyslipidemia, test_size=0.2, random_state=42, stratify=y_hypertension
)

# 標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ 資料準備完成")
print(f"訓練集: {X_train_scaled.shape[0]} 人")
print(f"測試集: {X_test_scaled.shape[0]} 人")
print(f"特徵數: {X_train_scaled.shape[1]} 個")

## 4. Genetic Programming 設定

**gplearn SymbolicClassifier 參數說明**：

**演化參數**：
- `population_size=1000`: 每一代的個體數量（族群大小）
- `generations=20`: 演化世代數（控制訓練時間）
- `tournament_size=20`: 競賽選擇的個體數（越大越傾向選擇好的個體）

**公式複雜度控制**：
- `init_depth=(2, 6)`: 初始公式深度範圍
- `init_method='half and half'`: 初始化方法（混合 full 和 grow）
- `function_set`: 允許的函數集合（加減乘除、log、sqrt、sin、cos 等）
- `parsimony_coefficient=0.01`: 懲罰複雜公式（避免過擬合）

**遺傳算子**：
- `p_crossover=0.7`: 交配機率
- `p_subtree_mutation=0.1`: 子樹突變機率
- `p_hoist_mutation=0.05`: 提升突變機率
- `p_point_mutation=0.1`: 點突變機率

**其他**：
- `metric='log loss'`: 分類使用 log loss
- `n_jobs=-1`: 使用所有 CPU 核心加速
- `verbose=1`: 顯示訓練進度

⏱️ **預計訓練時間**：每個模型 30-60 分鐘（取決於 generations）

In [ ]:
# 定義 GP 參數
gp_params = {
    'population_size': 1000,
    'generations': 20,
    'tournament_size': 20,
    'init_depth': (2, 6),
    'init_method': 'half and half',
    'function_set': ('add', 'sub', 'mul', 'div', 'log', 'sqrt', 'abs', 'neg', 'max', 'min'),
    'parsimony_coefficient': 0.01,
    'p_crossover': 0.7,
    'p_subtree_mutation': 0.1,
    'p_hoist_mutation': 0.05,
    'p_point_mutation': 0.1,
    'metric': 'log loss',
    'n_jobs': -1,
    'verbose': 1,
    'random_state': 42
}

print("✅ GP 參數設定完成")
print(f"族群大小: {gp_params['population_size']}")
print(f"演化世代: {gp_params['generations']}")
print(f"預計訓練時間: ~{gp_params['generations'] * 2}-{gp_params['generations'] * 3} 分鐘/模型")

## 5. 訓練 GP 模型

⚠️ **注意**: GP 訓練**非常慢**，請耐心等候！

如果時間不足，可以減少 `generations` 參數（例如改為 10）。

In [ ]:
# 訓練三個 GP 模型
diseases = ['高血壓', '高血糖', '高血脂']
y_trains = [y_train_hp, y_train_hg, y_train_dl]
y_tests = [y_test_hp, y_test_hg, y_test_dl]

gp_models = {}
gp_results = []

print("="*80)
print("Genetic Programming 訓練中...（這會很慢，請耐心等候）")
print("="*80)
print()

for disease, y_train, y_test in zip(diseases, y_trains, y_tests):
    print(f"\n{'='*80}")
    print(f"訓練 {disease} GP 模型...")
    print(f"{'='*80}")
    
    start_time = time.time()
    
    # 建立 GP 模型
    model = SymbolicClassifier(**gp_params)
    
    # 訓練
    model.fit(X_train_scaled, y_train)
    
    training_time = time.time() - start_time
    
    # 預測
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    # 儲存結果
    gp_models[disease] = model
    gp_results.append({
        '疾病': disease,
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall,
        '訓練時間(秒)': training_time,
        '訓練時間(分鐘)': training_time / 60,
        '公式長度': len(str(model._program)),
        '公式深度': model._program.depth()
    })
    
    # 輸出結果
    print(f"\n{disease} 訓練完成:")
    print(f"  訓練時間:  {training_time:.1f} 秒 ({training_time/60:.1f} 分鐘)")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  公式深度:  {model._program.depth()}")
    print(f"  公式長度:  {len(str(model._program))} 字元")
    
    # 混淆矩陣
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

print("\n" + "="*80)
print("✅ Genetic Programming 訓練完成！")
print("="*80)

## 6. GP 效能總結

In [ ]:
# 顯示 GP 結果
df_gp = pd.DataFrame(gp_results)

print("="*100)
print("Genetic Programming 效能總結")
print("="*100)
print(df_gp[['疾病', 'AUC', 'F1', 'Precision', 'Recall', '訓練時間(分鐘)', '公式深度']].to_string(index=False))

print(f"\n訓練時間統計:")
print(f"  總訓練時間: {df_gp['訓練時間(秒)'].sum():.1f} 秒 ({df_gp['訓練時間(分鐘)'].sum():.1f} 分鐘)")
print(f"  平均訓練時間: {df_gp['訓練時間(分鐘)'].mean():.1f} 分鐘")

## 7. 演化出的公式分析

In [ ]:
# 顯示演化出的公式
print("="*100)
print("演化出的預測公式")
print("="*100)

for disease in diseases:
    model = gp_models[disease]
    print(f"\n{disease}:")
    print(f"  深度: {model._program.depth()}")
    print(f"  長度: {len(str(model._program))} 字元")
    print(f"  公式: {str(model._program)}")
    print()

## 8. 與最佳模型比較

In [ ]:
print("="*80)
print("重新訓練基準模型作為比較")
print("="*80)
print()

# 準備 y_multi
y_train_multi = np.column_stack([y_train_hp, y_train_hg, y_train_dl])
y_test_multi = np.column_stack([y_test_hp, y_test_hg, y_test_dl])

# 訓練 MTL LR
mtl_lr = MultiOutputClassifier(
    LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
)
mtl_lr.fit(X_train_scaled, y_train_multi)

# 訓練 XGBoost
from xgboost import XGBClassifier

xgb_models = {}
for disease, y_train, y_test in zip(diseases, y_trains, y_tests):
    n_positive = np.sum(y_train == 1)
    n_negative = np.sum(y_train == 0)
    scale_pos_weight = n_negative / n_positive
    
    model = XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        max_depth=5,
        learning_rate=0.1,
        n_estimators=100,
        random_state=42,
        eval_metric='logloss'
    )
    model.fit(X_train_scaled, y_train)
    xgb_models[disease] = model

print("✅ 基準模型訓練完成！")

In [ ]:
# 計算所有模型的結果
all_results = []

# MTL LR
y_pred_lr = mtl_lr.predict(X_test_scaled)
for i, disease in enumerate(diseases):
    y_test = y_test_multi[:, i]
    y_pred = y_pred_lr[:, i]
    y_pred_proba = mtl_lr.estimators_[i].predict_proba(X_test_scaled)[:, 1]
    
    all_results.append({
        '方法': 'LR',
        '疾病': disease,
        'AUC': roc_auc_score(y_test, y_pred_proba),
        'F1': f1_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0)
    })

# XGBoost
for disease, y_test in zip(diseases, y_tests):
    model = xgb_models[disease]
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    all_results.append({
        '方法': 'XGBoost',
        '疾病': disease,
        'AUC': roc_auc_score(y_test, y_pred_proba),
        'F1': f1_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0)
    })

# GP
for result in gp_results:
    all_results.append({
        '方法': 'GP',
        '疾病': result['疾病'],
        'AUC': result['AUC'],
        'F1': result['F1'],
        'Recall': result['Recall']
    })

df_all = pd.DataFrame(all_results)

print("="*100)
print("所有模型效能比較 (含 GP)")
print("="*100)
print(df_all[['方法', '疾病', 'AUC', 'F1', 'Recall']].to_string(index=False))

## 9. 視覺化比較

In [ ]:
# 建立視覺化
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, metric in enumerate(['AUC', 'F1', 'Recall']):
    ax = axes[idx]
    
    # 準備資料
    pivot_data = df_all.pivot(index='疾病', columns='方法', values=metric)
    
    # 繪圖
    pivot_data.plot(kind='bar', ax=ax, width=0.8)
    ax.set_title(f'{metric} 比較 (含 GP)', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12)
    ax.set_xlabel('疾病', fontsize=12)
    ax.legend(title='方法', fontsize=10, loc='lower right')
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 1)
    
    # 添加數值標籤
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f', fontsize=8)

plt.tight_layout()
plt.savefig('../../docs/experiments/gp_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 視覺化完成，已儲存至 docs/experiments/gp_comparison.png")

## 10. 結論

### 🧬 Genetic Programming 效能總結

本實驗測試了 Genetic Programming (GP) 在三高疾病預測上的表現。

### 📊 GP 實際表現

**效能結果**：
- **高血壓**: AUC=?, F1=?, Recall=?, 訓練時間=? 分鐘
- **高血糖**: AUC=?, F1=?, Recall=?, 訓練時間=? 分鐘
- **高血脂**: AUC=?, F1=?, Recall=?, 訓練時間=? 分鐘

### 🏆 模型排名比較

（執行完成後填入實際排名）

### 🧬 演化公式分析

（執行完成後分析演化出的公式）

### 🔍 GP 的優缺點分析

**✅ 優點**：
- **最高可解釋性**: 演化出明確的數學公式
- **特徵交互發現**: 自動找到特徵組合
- **無需假設模型形式**: 完全從資料中學習
- **符號回歸**: 可用於科學發現

**❌ 缺點**：
- **訓練極慢**: 需要演化多個世代
- **易過擬合**: 公式可能過於複雜
- **難以調參**: 演化參數多且敏感
- **計算資源需求大**: CPU 密集型

### 💡 最終建議

（執行完成後根據實際結果提供建議）

---

**下一步**: 建立完整的模型比較總結表